In [1]:
from muselsl import *
import time
import numpy as np
import serial
from pylsl import StreamInlet, resolve_byprop
import warnings
from psycho_tracker import Calibrator
import utils 
import xml.dom.minidom
from data_processor import *
from settings import *
from metrics import *
from PsychoPy_Code.PsychoRun import run_psychopy

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Search for active LSL streams

print('Looking for an EEG stream...')
streams = resolve_byprop('type', 'EEG', timeout=2)
if len(streams) == 0:
    raise RuntimeError('Can\'t find EEG stream.')

# Set active EEG stream to inlet and apply time correction

print("Start acquiring data")
inlet = StreamInlet(streams[0], max_chunklen=12)
#eeg_time_correction = inlet.time_correction()

# Get the stream info and description
info = inlet.info()

Looking for an EEG stream...
Start acquiring data


In [8]:
"""

2. Obtain Threshold from PsychoPy 
"""

cal = Calibrator(inlet=inlet, info=info, channel_index=Channel.FP1)

cal.start_stage(mode='relax', stage=1)
time.sleep(1)

cal.end_stage(mode='relax', stage=1)

#run_psychopy(cal=cal)
#cal.update_stage_threshold()

NameError: name 'Channel' is not defined

In [6]:
""" 
1. CONNECT TO EEG STREAM 
"""
threshold = 0.0223716588909309062
print("FINAL THRESH", threshold)

"""
3. Record data and wave hand if metric is above threshold.  

"""
fs = utils.get_fs(info)
print('Press Ctrl-C in the console to break the while loop.')

#arduinoData = serial.Serial('com3',9600)
time.sleep(1)
try:
    
    dp = DataProcessor(fs=fs)
    sec = 1
    count = sec * fs

    while True:
        """ 3.1 ACQUIRE DATA """

        # Obtain EEG data from the LSL stream
        eeg_data, timestamp = utils.acquire_eeg_data(inlet, fs)
        
        dp.feed_new_data(eeg_data=eeg_data)
        dp.append_metrics()
        
        # For capturing mock data
        count -= 1
        if count <= 0:
            break
            
    df = dp.get_dataframe()
    print(df)
    print(df.to_csv())
    
        
#         for i in range(NUM_CHANNELS):  # Iterate through all separate channels

#             # Record channel smooth band power
#             csbp = c.get_channel_smooth_band_powers(i) 
#             # Run calculations on csbp to obtain desired metrics

#             if i in [1,2,3]:
#                 theta_metric = csbp[1]

#                 metrics.append(theta_metric)
#                 #ratios.append(Metrics.beta_protocol(csbp, Band))
#                 print(theta_metric)
#         metric = np.mean(metrics)  
#         print(metric)    


#         if check_positive(_metric=metric, _threshold=threshold): 

#             hand_wave()
#         else:
#             hand_stay()

except KeyboardInterrupt:
    print('Closing!')

#arduinoData.close()

FINAL THRESH 0.022371658890930905
Press Ctrl-C in the console to break the while loop.
     (DRL, AB)  (DRL, AD)  (DRL, ALPHA)  (DRL, AT)  (DRL, BETA)  (DRL, BT)  \
0     1.952132   0.462227      0.031032   0.737996     0.015897   0.378046   
1     1.454320   0.443919      0.029034   0.701269     0.019964   0.482197   
2     1.211584   0.483626      0.032364   0.700764     0.026712   0.578387   
3     1.615535   0.411430      0.027125   0.615517     0.016790   0.380999   
4     1.573632   0.422507      0.028886   0.683382     0.018356   0.434270   
5     1.517201   0.346854      0.022889   0.574603     0.015086   0.378726   
6     0.864114   0.256378      0.017609   0.456263     0.020378   0.528013   
7     1.204907   0.225480      0.015403   0.443012     0.012784   0.367673   
8     1.234019   0.389447      0.025311   0.609476     0.020511   0.493896   
9    -0.366047  -0.085304     -0.005191  -0.187934     0.014182   0.513414   
10    1.418579   0.482312      0.029662   1.058853     

In [64]:
arduinoData.close()